# ディープラーニングの実装基礎
1. ディープラーニングフレームワークの１つであるKerasの簡単な使い方を学んでみよう
1. パラメータをチューニングして、予測精度の変化を確認しよう
1. Data Augmentationを追加して、更に予測精度を改善してみよう

### おまじない

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

from skimage import io

### マスタデータを読み込んでみよう
- 今回は[【練習問題】画像ラベリング（10種類）コンペ](https://signate.jp/competitions/133#abstract)に挑戦します
- まずは画像データのラベルデータである`train_master.tsv`と`label_master.tsv`及び応募用サンプルファイル`sample_submit.tsv`を読み込みます

In [ ]:
labels = pd.read_csv("train_master.tsv", sep="\t")
master = pd.read_csv("label_master.tsv", sep="\t")
sample = pd.read_csv("sample_submit.tsv", header=None, sep="\t")
print(labels.shape)
print(master.shape)
print(sample.shape)

### マスタデータの中身を確認してみよう

In [ ]:
labels.head()

In [ ]:
master

In [ ]:
sample.head()

### 画像データを読み込んでみよう
- `train_images`フォルダと`test_images`フォルダの中にある画像を全て読み込みましょう
- ファイルの読み込む順番に気を付けましょう
    - 何も考えずにos.listdir関数等で全てのファイル名を取ってきてから読み込むとlabelsデータの順番と不整合になります
        - e.g) train_0.jpg, train_1.jpg, train_10.jpg, train_100.jpg ...
    - その為、train_imagesからはlabelsデータから1行ずつファイル名（file_name）を取り出しながら読み込みを行います
    - test_imagesも同様な理由で、sampleデータから1行ずつファイル名を取り出しながら読み込みを行います
- 読み込んだ画像はそれぞれ`train_imgs`と`test_imgs`という変数に代入し、データ型はnumpyのarrayに変換しておきましょう

In [ ]:
train_imgs = []
for fname in labels["file_name"]:
    path = "./train_images/" + fname
    img = io.imread(path)
    train_imgs.append(img)
print(type(train_imgs))
train_imgs = np.array(train_imgs)
print(type(train_imgs), train_imgs.shape)

In [ ]:
test_imgs = []
for fname in sample[0]:
    path = "./test_images/" + fname
    img = io.imread(path)
    test_imgs.append(img)
print(type(test_imgs))
test_imgs = np.array(test_imgs)
print(type(test_imgs), test_imgs.shape)

### 読み込んだ画像を可視化してみよう
- 2行5列で読み込んだ画像を表示させましょう
- 各画像のtitleには画像に付与されたラベルを付けます
    - 手順としては変数labelsからi行目のlabel_idを取り出し、変数masterからlabel_idが一致するlabel_nameを取得します

In [ ]:
plt.figure(figsize=(12,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    target_id = labels.loc[i,"label_id"] # target画像のlabel_idを取得
    label = master[master["label_id"]==target_id]["label_name"].values[0] # masterからlabel_idに対応するlabel_nameを取得
    plt.title(label)
    plt.imshow(train_imgs[i])

### ラベルの分布を確認しておきましょう

In [ ]:
labels["label_id"].value_counts()

### 画像データの画素値を正規化しておきましょう
- DeepLearningで画像を学習させる前準備として画素値を正規化しておきましょう
- 現在は画素値の値域が0~255である為、0~1となるように正規化をしておきます
- 正規化をすることで、DeepLearningの学習を円滑に行う効果を期待できます

In [ ]:
train_imgs

In [ ]:
train_imgs = train_imgs / 255
test_imgs = test_imgs / 255

In [ ]:
train_imgs

### 目的変数であるラベルデータをone-hot-encodingしておきましょう
- 今回は画像に付与される10種類のラベルを予測する問題です
- この場合、モデルの出力は各ラベルに対する確率値となります
    - output = [label1の確率, label2の確率, label3の確率, ..., label10の確率]
- その為、学習データもこの形に合わせる必要がある為、ラベルデータをone-hot-encodingしましょう
    - e.g) label2の場合は[0,1,0,0,0,0,0,0,0,0]とする
- one-hot-encodingはkerasのutil.to_categorical関数が便利なので、こちらを利用します

In [ ]:
y = labels["label_id"]
y.head()

In [ ]:
from keras import utils
y_categorical = utils.to_categorical(y)

In [ ]:
y_categorical

### 学習パラメータの評価の為に、学習データを２つに分割しましょう
- モデルのパラメータをチューニングする際には、学習データを構築/検証データに分割するという方策をとります
- 構築データでモデルを学習し、学習に利用しなかった検証データを未知データとし、モデルの汎化性能を評価します
- numpyではnp.split関数を使うことで、簡単にデータを分割することができます
- 具体的には、`np.split(元の配列, [何番目で分割するかの数値])`と記述します
    - より詳しくは例えば[こちら](https://note.nkmk.me/python-numpy-split/)を参考にして下さい

In [ ]:
np.split([1,2,3,4,5],[3])

- 今回は学習データ5000枚中、4000枚を構築データ、残り1000枚を検証データとします
- 構築用画像を`X_tr`、検証用画像を`X_val`としましょう
- 同様に、構築用画像のラベルを`y_tr`、検証用画像のラベルを`y_val`としましょう

In [ ]:
X_tr, X_val = np.split(train_imgs,[4000])
y_tr, y_val = np.split(y_categorical, [4000])

In [ ]:
print(X_tr.shape, X_val.shape)
print(y_tr.shape, y_val.shape)

### Kerasの使い方の基本を学びましょう
- 基本となる使い方は下記となります
    1. `model = Sequential()`：モデルの箱を準備する
    1. `model.add(XXX)`：add関数の中にモジュールを記述することで、Neural Networkの構造を追加していく
    1. `model.compile(loss="xxx", optimizer="yyy", metrics=["zzz"])`：損失関数やオプティマイザー、評価関数等を設定
    1. `model.fit(trainX, y, batch_size, epoch, verbose=1, validation_data=(X_val, y_val))`：学習データやbatch_size, epoch等を設定して学習


- 今回利用するモジュールは下記です
    - Dense(X)：全結合層であり、Xはノード数を表す
    - Conv2D（filters=X, kernel_size=(x,x), padding）：畳み込み層であり、Xはフィルタの数、(x,x)はフィルタの大きさを表す
    - MaxPooling2D(pool_size=(x,x))：プーリング（max pooling）層であり、(x,x)はプーリング窓の大きさを表す
    - Activation("xxx")：活性化関数を表し、xxxに活性化関数（sigmoidやrelu）の前を記述する
    - Flatten()：多次元配列を1次元配列に変換する。畳み込み層から全結合層へ接続する際に必要となる

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

### 練習で下記構造をKerasで作ってみましょう
- Sequentialを使い、モデルの箱を準備し、モデルに層を追加してみましょう

<img src="MLP.png" width="300">

### CNNを使って実際に学習をしてみよう

- model作成部分

In [ ]:
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(3,3), padding="same", input_shape=(96,96,3)))
model.add(Activation("sigmoid"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=12, kernel_size=(3,3), padding="same"))
model.add(Activation("sigmoid"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=120))
model.add(Activation("sigmoid"))
model.add(Dense(units=60))
model.add(Activation("sigmoid"))
model.add(Dense(units=10))
model.add(Activation("softmax"))

In [ ]:
model.summary()

- 学習に必要となる損失関数やオプティマイザーと、学習結果の確認の為の評価関数の設定
    - SGDの細かいパラメータ詳細については[こちら](https://keras.io/ja/optimizers/)
    - lr（学習率）が特に重要なパラメータです

In [ ]:
model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.SGD(lr=0.5, momentum=0.9, decay=0.0, nesterov=True),
             metrics=["accuracy"])

- batch_sizeとepochの設定

In [ ]:
batch_size=100
epochs=10

- モデルの学習

In [ ]:
history = model.fit(X_tr, y_tr,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val))

### 学習曲線の可視化してみよう
- 損失関数と評価関数の遷移を見ることで、モデルの学習が進んでいるのかや、過学習していないかを確認することができます

In [ ]:
def learning_plot(history, epochs):
    fig = plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.plot(range(1,epochs+1), history.history['loss'])
    plt.plot(range(1,epochs+1), history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.xticks(range(1,epochs+1))
    plt.ylabel('loss')
    plt.legend(['train', 'val'], loc='upper right')
    plt.subplot(1,2,2)
    plt.plot(range(1,epochs+1), history.history['acc'])
    plt.plot(range(1,epochs+1), history.history['val_acc'])
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.xticks(range(1,epochs+1))
    plt.ylabel('accuracy')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
learning_plot(history,epochs)

### 学習率を0.05にして再度モデルを学習し直してみよう
- 学習が進んでいないのは学習率が高すぎる可能性があります

In [ ]:
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(3,3), padding="same", input_shape=(96,96,3)))
model.add(Activation("sigmoid"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=12, kernel_size=(3,3), padding="same"))
model.add(Activation("sigmoid"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=120))
model.add(Activation("sigmoid"))
model.add(Dense(units=60))
model.add(Activation("sigmoid"))
model.add(Dense(units=10))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.SGD(lr=0.05, momentum=0.9, decay=0.0, nesterov=True),
             metrics=["accuracy"])

In [ ]:
batch_size=100
epochs=20

In [ ]:
history = model.fit(X_tr, y_tr,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val))

In [ ]:
learning_plot(history,epochs)

### 演習①　活性化関数をsigmoidからreluに変換し、学習をし直してみましょう

### 作ったモデルを使い、test_imgsの画像へのラベル予測を行い、SIGNATEに投稿してみましょう
- 予測は`model.predict`関数を使います
- 予測結果はpred = [label1の確率, label2の確率, ..., label10の確率]と返ってきますが、label_idを1つに決める必要があります
- その為、`argmax(axis=1)`関数を使い、一番確率値が高いindexを取得します（indexはlabel_idと一致している為）

In [ ]:
pred = model.predict(test_imgs)
pred

In [ ]:
pred = pred.argmax(axis=1)
pred

In [ ]:
sample[1] = pred
sample.to_csv("submit1.tsv", sep="\t", index=None, header=None)

### 演習②　今度はオプティマイザーをAdamに変更してモデルを学習し、予測結果をSIGNATEに投稿しましょう
- Adamを使う為には、`optimizer.Adam()`と記述します
- 投稿ファイルは`submit2.tsv`というファイル名で保存しましょう

### 演習③　層を１つ又は２つ程度追加し、精度を検証してみましょう
- 良いパラメータがあれば、SIGNATEにも投稿してみましょう
- filtersの値やkernel_sizeの値も変更してみましょう

### 精度の検証をしてみよう
- 今回作成したモデルがどんな画像に対して当たっていて、どんな画像を外しているのかを確かめることも重要です
- 今回のようにラベル自体を出力結果とする時には混合行列を算出し、確かめることをします
- まず、検証用データに対する予測値を求め、sklearnのconfusion_matrix関数を使い、混合行列を求めます

In [ ]:
pred = model.predict(X_val)
pred = pred.argmax(axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val.argmax(axis=1), pred)
cm

- 続いて生成したconfusion_matrixを見やすくするためにヒートマップを描きます

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]))
    ax.set_title(title, size=20)
    ax.set_ylabel('True label', fontsize=15)
    ax.set_xlabel('Predicted label', fontsize=15)
    ax.set_xticklabels(classes, fontsize=15)
    ax.set_yticklabels(classes, fontsize=15)
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right',
             rotation_mode='anchor')

    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), fontsize=20,
                    ha='center', va='center',
                    color='white' if cm[i, j] > thresh else 'black')
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(cm, master["label_name"].values)
plt.show()

In [ ]:
def plot_img(true_label, pred_label,pred,y_val):
    tmp = pd.DataFrame({"pred":pred,"true":y_val.argmax(axis=1)})
    tix = master[master["label_name"]==true_label]["label_id"].values[0]
    pix = master[master["label_name"]==pred_label]["label_id"].values[0]
    tmp = tmp[(tmp["pred"]==pix)&(tmp["true"]==tix)]
    
    if tmp.shape[0] > 0:    
        n = int(np.ceil(np.sqrt(tmp.shape[0])))
        fig = plt.figure(figsize=(n+5,n+5))
        fig.subplots_adjust(left=0, right=0.5, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
        for i,v in enumerate(tmp.index):
            ax = fig.add_subplot(n, n, i + 1, xticks=[], yticks=[])
            ax.imshow(X_val[v])

In [ ]:
plot_img(true_label="dog", pred_label="cat", pred=pred, y_val=y_val)

In [ ]:
plot_img(true_label="cat", pred_label="dog", pred=pred, y_val=y_val)

### ラベル別の正解率を算出してみよう
- confusion_matrixを使えば、ラベル別に正解率を見ることもできます
- diagonal関数は対角成分（つまり各ラベルで正解した数）のみを抽出することができます
- 従って「ラベルの正解数÷ラベルの総数」により、各ラベルの正解率を求めることができます

In [ ]:
acc = cm.diagonal()/cm.sum(axis=1)
ac1 = pd.DataFrame({"label":master["label_name"], "Accuracy":acc})
ac1

### モデルの保存の仕方とモデルの読み込み方について知ろう
- モデルはHDF5というファイルとして保存される為、拡張子は「.h5」とします
- 保存は`model.save()`、読み込みは`load_model()`を使います

In [ ]:
file_path = 'my_model.h5'
model.save(file_path)

In [ ]:
from keras.models import load_model

model = load_model(file_path)

## ▼更に精度向上を狙い、Data Augmentationを試してみよう
- Data Augmentationとはデータを水増しすることで、画像のパターンを増やして汎化性能を上げることを狙った手法を言います
- Data Augmentationは学習データに対する水増しと、予測データに対する水増しの2種類あります。
    - 後者はtest time augmentation（TTA）と呼ばれます
- 今回は前者の学習データに対する水増しの方法を学びましょう

### kerasのライブラリを使い、データ水増し器を作ろう
- scikit-image等を使いデータ水増しをすることもできますが、簡単な画像処理であればkerasに便利なライブラリ「ImageDataGenerator」があります

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

### ジェネレーターの使い方を学ぼう
- 基本的な画像操作の指定方法は下記です（より詳細は[こちら](https://keras.io/ja/preprocessing/image/)）
    1. 平行移動：height_shift_rangeとwidth_shift_rangeに平行移動させる割合（0.0-1.0）を設定する
    1. 反転：上下ならvertical_flip、左右ならhorizotal_flipをTrueにする
    1. 回転：rotation_rangeに最大回転角度の数値を設定する
    

- ジェネレーターの使い方は下記のようになります
    1. 画像操作の内容を指定したジェネレーターを宣言する
    1. flow関数を使い、iteratorを作る。その際、batch_sizeの指定が必要

In [ ]:
img = train_imgs[1:2,]
batch_size = 1

# 平行移動
g1 = ImageDataGenerator(height_shift_range=0.5, width_shift_range=0.5)
i1 = g1.flow(img, batch_size=batch_size)

# 左右反転
g2 = ImageDataGenerator(horizontal_flip=True)
i2 = g2.flow(img, batch_size=batch_size)

# 回転
g3 = ImageDataGenerator(rotation_range=90)
i3 = g3.flow(img, batch_size=batch_size)

In [ ]:
fig, ax = plt.subplots(3,5,figsize=(10,5))
i = 0
for a,b,c in zip(i1,i2,i3):
    if i > 4:
        break
    ax[0][i].imshow(a[0])
    ax[1][i].imshow(b[0])
    ax[2][i].imshow(c[0])
    i+=1
ax[0][4].text(105,55,"<- shift images",fontsize=20)
ax[1][4].text(105,55,"<- flip images",fontsize=20)
ax[2][4].text(105,55,"<- rotate images",fontsize=20)
plt.show()

### Data Augmentationを使い、モデルを学習しましょう

In [ ]:
model = Sequential()
model.add(Conv2D(filters=12, kernel_size=(3,3), padding="same", input_shape=(96,96,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=24, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=36, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=120))
model.add(Activation("relu"))
model.add(Dense(units=60))
model.add(Activation("relu"))
model.add(Dense(units=10))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.Adam(),
             metrics=["accuracy"])

In [ ]:
batch_size=100
epochs=60

- Data AugumentationのGeneratorを生成
    - ここでは`height_shift_range=0.2, width_shift_range=0.2, horizontal_flip=True`とします

In [ ]:
# 構築データ用のジェネレーター
datagen = ImageDataGenerator(height_shift_range=0.2, width_shift_range=0.2, horizontal_flip=True)
iterator = datagen.flow(X_tr, y_tr, batch_size=batch_size)

# 検証データ用のジェネレーター（特に何もしない）
datagen_val = ImageDataGenerator()
iterator_val = datagen_val.flow(X_val, y_val, batch_size=1, shuffle=False)

- Generatorを使い学習をする場合には、fit関数ではなく、fit_generator関数を代わりに使います
- この時、steps_per_epochを指定する必要がありますが、これは1epochあたり何ステップ実施するかを意味します
    - 通常はデータサイズ÷バッチサイズとします
    - batch_size=100ならば、1stepあたり100枚学習に使うことになるので、4000枚の学習画像全てを網羅する為には4000/100=40回必要になる為

In [ ]:
history = model.fit_generator(generator=iterator, 
                              steps_per_epoch=len(X_tr)/batch_size, 
                              epochs=epochs, 
                              validation_data = iterator_val,
                              validation_steps = len(X_val))

In [ ]:
learning_plot(history,epochs)

### 精度の検証をしてみよう

In [ ]:
pred = model.predict(X_val)
pred = pred.argmax(axis=1)
cm = confusion_matrix(y_val.argmax(axis=1), pred)

In [ ]:
plot_confusion_matrix(cm, master["label_name"].values)
plt.show()

In [ ]:
plot_img(true_label="dog", pred_label="cat", pred=pred, y_val=y_val)

In [ ]:
plot_img(true_label="cat", pred_label="dog", pred=pred, y_val=y_val)

In [ ]:
acc = cm.diagonal()/cm.sum(axis=1)
ac2 = pd.DataFrame({"label":master["label_name"], "Accuracy":acc})
ac2

In [ ]:
ac1

### 応募用ファイルを作り、SIGNATEに投稿してみよう

In [ ]:
pred = model.predict(test_imgs)
pred = pred.argmax(axis=1)
sample[1] = pred
sample.to_csv("submit4.tsv", sep="\t", index=None, header=None)